In [ ]:
import ee
ee.Initialize()
import pandas as pd

In [ ]:
tile_file = "gee_tile_select.csv"
data = pd.read_csv(tile_file)
minx = data['left']
maxy = data['top']
maxx = data['right']
miny = data['bottom']
tile_id =data["id"]
size_data = len(minx)

In [ ]:
def start_process(i):
    #define start data and end date, get area exgtent for each tile
    s_date = '2015-01-01'
    e_date = '2016-01-01'
    area = ee.Geometry.Rectangle([minx[i], maxy[i], maxx[i], miny[i]])
    tile_name = 'plantation_' + str(tile_id[i])
    
    #entropy
    #Define a neighborhood with a kernel.
    entropy_args = {'radius': 4}
    square = ee.Kernel.square(**entropy_args)
    
    #define image objects
    gain = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('gain')
    vcf = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('treecover2000')
    lossyear = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('lossyear')
    landsat = ee.Image('UMD/hansen/global_forest_change_2015').select('last_b30','last_b40','last_b50','last_b70')
    cropDominance = ee.Image('USGS/GFSAD1000_V0').select('landcover')   
    nir = landsat.select('last_b30')
    nir_entropy = nir.entropy(square)

    zones = gain.eq(1).And((lossyear.lt(12)).Or(lossyear.gt(16)))
    zones = zones.updateMask(zones)
    
    #reduce raster to polygons
    reduce_combine_args = {
      'reducer2': ee.Reducer.stdDev(),
      'sharedInputs': True
    }
    reducers = ee.Reducer.mean().combine(**reduce_combine_args);
    
    reduce_args = {
      'geometry': area,
      'crs': gain.projection(),
      'scale': 30,
      'geometryType': 'polygon',
      'eightConnected': True,
      'labelProperty': 'regrowth',
      'reducer': reducers,
      'bestEffort': True,
      'maxPixels': 1e18
    }

    vectors = zones.addBands(landsat).addBands(vcf).addBands(lossyear).addBands(cropDominance).addBands(nir_entropy).reduceToVectors(**reduce_args)
   
    featureCollection = ee.FeatureCollection([vectors]).flatten()
    
    down_args = {
    'collection': featureCollection,
    'folder': 'plantation_landsat_2015_v3',
    'description': tile_name,
    'fileFormat': 'KML'
    }
    task = ee.batch.Export.table.toDrive(**down_args)
    task.start()

In [ ]:
#loop through tiles
for i in range(0,size_data):
    start_process(i)